In [ ]:
import json
import boto3
import pymysql

# S3 및 RDS 설정
s3 = boto3.client("s3")
rds_host = "your_rds_endpoint"
username = "your_db_username"
password = "your_db_password"
database_name = "your_database_name"


# Lambda 핸들러 함수
def lambda_handler(event, context):
    # S3에서 데이터 가져오기
    response = s3.get_object(Bucket="your_bucket_name", Key="response_data.json")
    data = json.loads(response["Body"].read())

    # RDS에 연결
    connection = pymysql.connect(
        host=rds_host, user=username, password=password, db=database_name
    )

    with connection.cursor() as cursor:
        for entry in data:
            student_id = entry["student_id"]
            session_id = entry["session_id"]
            question_id = entry["question_id"]
            response = entry["response"]

            # RDS에서 해당 question_id의 문항모수를 가져와서 채점
            cursor.execute(
                "SELECT a, b, c FROM question_bank WHERE question_id = %s",
                (question_id,),
            )
            question_params = cursor.fetchone()

            if question_params:
                a, b, c = question_params
                # 예시: 3PL 모델을 사용하여 정답 확률 계산 및 채점
                probability_correct = c + (1 - c) / (1 + np.exp(-1.7 * a * (theta - b)))
                is_correct = int(response == 1 if probability_correct > 0.5 else 0)

                # SageMaker 호출하여 θ 값 업데이트 (생략: SageMaker 호출 코드 추가 필요)

                # RDS에 학생의 θ 값 업데이트
                cursor.execute(
                    "UPDATE student_abilities SET theta = %s WHERE student_id = %s",
                    (new_theta, student_id),
                )

        # 변경 사항 커밋
        connection.commit()

    connection.close()
    return {"statusCode": 200, "body": json.dumps("Processing completed successfully.")}

# 트리거

# Lambda 함수 트리거 설정

AWS Lambda 함수가 특정 이벤트에 따라 자동으로 실행되도록 설정하려면, 아래와 같은 트리거 조건 및 설정 방법을 사용합니다.

## 1. 초기 데이터 처리 Lambda 트리거

- **트리거 조건**: 학생이 처음 접속할 때 (초기 데이터 요청이 있을 때) 트리거됩니다.
- **설정 방법**:
  - API Gateway를 트리거로 설정합니다.
  - 학생이 초기 접속 시 API Gateway에 요청을 보내면, 이 요청이 Lambda 함수로 전달됩니다.
  - API Gateway에서 HTTP POST 메서드로 특정 경로 (`/initialize`)를 설정하여 초기 데이터를 처리하는 Lambda 함수가 트리거되도록 합니다.

## 2. 응답 데이터 처리 Lambda 트리거

- **트리거 조건**: 학생이 문제에 대한 응답을 제출할 때마다 트리거됩니다.
- **설정 방법**:
  - API Gateway를 트리거로 설정합니다.
  - 학생이 문제에 대한 응답을 제출하면, API Gateway에서 해당 요청을 응답 데이터 처리 Lambda 함수로 전달합니다.
  - 예를 들어, API Gateway에서 HTTP POST 메서드로 `/submit-response` 경로를 설정하고, 이 경로로 요청이 들어올 때마다 응답 데이터 처리 Lambda가 실행되도록 합니다.

## 3. 문항 선택 및 제공 Lambda 트리거

- **트리거 조건**: 새 문제를 요청할 때 트리거됩니다.
- **설정 방법**:
  - API Gateway를 트리거로 설정하여, 학생이 새 문제를 요청하면 API Gateway에서 이 요청을 문항 선택 및 제공 Lambda 함수로 전달합니다.
  - API Gateway에서 HTTP GET 메서드로 `/get-next-question` 경로를 설정하여, 이 경로로 요청이 들어올 때마다 문항 선택 및 제공 Lambda가 실행되도록 합니다.

## 4. 검사 종료 평가 Lambda 트리거

- **트리거 조건**: 응답 데이터 처리 후 특정 조건 (예: θ 값 변동이 일정 범위 내로 유지됨)에 도달하면, 검사 종료 여부를 평가합니다.
- **설정 방법**:
  - **다른 Lambda 함수에서 직접 호출**: 응답 데이터 처리 Lambda가 θ 값 계산을 마친 후, θ 값이 특정 조건을 만족하는지 확인하고, 조건이 충족되면 검사 종료 평가 Lambda를 호출하도록 설정합니다.
  - **Lambda 내의 조건부 호출**: 응답 데이터 처리 Lambda 내에서 검사 종료 조건을 평가하고, 조건이 충족되었을 때 검사 종료 평가 Lambda를 호출하여 종료 여부를 처리하도록 합니다.
  - 이를 통해 Lambda 함수 간의 호출로 트리거할 수 있습니다.

## 전체 트리거 설정 예시

### API Gateway 트리거 설정

1. **API Gateway 콘솔로 이동**합니다.
2. 새로운 **REST API 또는 HTTP API**를 생성합니다.
3. 리소스 경로를 추가하고, 각 Lambda 함수에 맞는 경로를 설정합니다:
   - `/initialize`: 초기 접속 시 Lambda 함수로 연결
   - `/submit-response`: 응답 데이터 처리 Lambda 함수로 연결
   - `/get-next-question`: 다음 문제 제공 Lambda 함수로 연결
4. 메서드를 선택 (POST, GET 등)하고, 각 메서드에서 Lambda 함수를 트리거하도록 연결합니다.

### Lambda 간 호출 설정

- **Lambda 함수 코드 내에서 다른 Lambda 함수 호출**:
  - `boto3` 라이브러리의 `lambda.invoke()` 메서드를 사용하여, 특정 조건을 만족할 때 다른 Lambda 함수를 호출합니다.
  - 예를 들어, 응답 데이터 처리 Lambda에서 검사 종료 조건을 확인하고, 조건이 충족되면 검사 종료 평가 Lambda를 호출합니다.

In [ ]:
import pymysql
import json

# RDS 설정
rds_host = "your_rds_endpoint"
username = "your_db_username"
password = "your_db_password"
database_name = "your_database_name"


def lambda_handler(event, context):
    student_id = event["student_id"]
    session_id = event["session_id"]

    # 초기 θ 값 설정
    initial_theta = 0.0  # 기본값으로 설정

    # RDS 연결
    connection = pymysql.connect(
        host=rds_host, user=username, password=password, db=database_name
    )

    with connection.cursor() as cursor:
        # 초기 θ 값과 세션 정보 저장
        cursor.execute(
            """
            INSERT INTO student_abilities (student_id, session_id, theta)
            VALUES (%s, %s, %s)
        """,
            (student_id, session_id, initial_theta),
        )
        connection.commit()

    connection.close()

    return {
        "statusCode": 200,
        "body": json.dumps("Initial data processed successfully."),
    }

In [ ]:
import json
import pymysql
import math

rds_host = "your_rds_endpoint"
username = "your_db_username"
password = "your_db_password"
database_name = "your_database_name"


def lambda_handler(event, context):
    student_id = event["student_id"]
    question_id = event["question_id"]
    response = event["response"]

    # RDS 연결
    connection = pymysql.connect(
        host=rds_host, user=username, password=password, db=database_name
    )

    with connection.cursor() as cursor:
        # 문항의 모수 가져오기 (a, b, c)
        cursor.execute(
            "SELECT a, b, c FROM question_bank WHERE question_id = %s", (question_id,)
        )
        question_params = cursor.fetchone()

        if question_params:
            a, b, c = question_params

            # 기존 θ 값 가져오기
            cursor.execute(
                "SELECT theta FROM student_abilities WHERE student_id = %s",
                (student_id,),
            )
            theta = cursor.fetchone()[0]

            # 3PL 모델을 사용하여 정답 확률 계산
            probability_correct = c + (1 - c) / (1 + math.exp(-1.7 * a * (theta - b)))

            # 응답에 따라 θ 업데이트 (간단한 예제)
            if response == 1:
                new_theta = theta + 0.1 * (1 - probability_correct)
            else:
                new_theta = theta - 0.1 * probability_correct

            # 업데이트된 θ 값 RDS에 저장
            cursor.execute(
                "UPDATE student_abilities SET theta = %s WHERE student_id = %s",
                (new_theta, student_id),
            )
            connection.commit()

    connection.close()

    return {
        "statusCode": 200,
        "body": json.dumps("Response processed and theta updated."),
    }

In [ ]:
import json
import pymysql

rds_host = "your_rds_endpoint"
username = "your_db_username"
password = "your_db_password"
database_name = "your_database_name"


def lambda_handler(event, context):
    student_id = event["student_id"]

    # RDS 연결
    connection = pymysql.connect(
        host=rds_host, user=username, password=password, db=database_name
    )

    with connection.cursor() as cursor:
        # 현재 학생의 θ 값 가져오기
        cursor.execute(
            "SELECT theta FROM student_abilities WHERE student_id = %s", (student_id,)
        )
        theta = cursor.fetchone()[0]

        # θ 값에 따라 적합한 문제 선택 (간단한 예시)
        cursor.execute(
            "SELECT question_id FROM question_bank ORDER BY ABS(theta_target - %s) LIMIT 1",
            (theta,),
        )
        question = cursor.fetchone()

        if question:
            question_id = question[0]
        else:
            question_id = None

    connection.close()

    return {"statusCode": 200, "body": json.dumps({"next_question_id": question_id})}

In [ ]:
import json
import pymysql

rds_host = "your_rds_endpoint"
username = "your_db_username"
password = "your_db_password"
database_name = "your_database_name"


def lambda_handler(event, context):
    student_id = event["student_id"]

    # RDS 연결
    connection = pymysql.connect(
        host=rds_host, user=username, password=password, db=database_name
    )

    with connection.cursor() as cursor:
        # 최근 θ 값 기록 가져오기
        cursor.execute(
            "SELECT theta FROM student_abilities WHERE student_id = %s ORDER BY timestamp DESC LIMIT 5",
            (student_id,),
        )
        theta_values = [row[0] for row in cursor.fetchall()]

        # 변동성 평가 (예: 최근 θ 값이 일정 범위 내에서 변동이 없는지 확인)
        if len(theta_values) == 5 and max(theta_values) - min(theta_values) < 0.05:
            is_finished = True
        else:
            is_finished = False

    connection.close()

    return {"statusCode": 200, "body": json.dumps({"is_finished": is_finished})}